In [103]:
import os
import pandas as pd

# Set the base directory
base_dir = './Data/'

Accidents = []

# Loop over the directories
for year in ['2019', '2020', '2021']:
    # Set the current directory
    current_dir = base_dir + year + '/'


    # Loop over the files in the current directory
    for file in os.listdir(current_dir):
        dfusagers = pd.read_csv(current_dir + "usagers-" + year + ".csv", sep=";")
        dfcaracteristiques = pd.read_csv(current_dir + "caracteristiques-" + year + ".csv", sep=";")
        dflieux = pd.read_csv(current_dir + "lieux-" + year + ".csv", sep=";")
        dfvehicles = pd.read_csv(current_dir + "vehicules-" + year + ".csv", sep=";")
       


        #join data frames for a given year
        result = pd.merge(dfusagers, dfcaracteristiques, on='Num_Acc')
        result = pd.merge(result, dflieux, on='Num_Acc')
        result = pd.merge(result, dfvehicles,  on=['Num_Acc','id_vehicule'])
    
        Accidents.append(result)




In [104]:
dfAccidents = pd.concat([Accidents[0], Accidents[1]])
dfAccidents = pd.concat([dfAccidents, Accidents[2]])
